<a href="https://colab.research.google.com/github/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 依使用者輸入：confirm, buttons, carousel, imgCarousel，分別傳送確認模板、按鈕模板、輪播模板及圖片輪播模板。
- 模板上的按鈕能觸發不同類型的 Action Object。

In [1]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pyngrok import ngrok
from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate, CarouselColumn, URIAction
)

# 讀取 secret
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
LINE_CHANNEL_ACCESS_TOKEN = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = userdata.get('LINE_CHANNEL_SECRET')

app = Flask(__name__)

configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text

        if action == 'confirm':
            template = ConfirmTemplate(
                text='你喜歡葬送的芙莉蓮嗎?',
                actions=[
                    MessageAction(label='是', text='我超喜歡芙莉蓮'),
                    MessageAction(label='否', text='其實我非常喜歡，只是我要傲嬌的說不')
                ]
            )
            reply = TemplateMessage(
                alt_text='這是確認視窗',
                template=template
            )

        elif action == 'carousel':
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/1/14/Summer_Wars_poster.jpg',
                        title='夏日大作戰',
                        text='細田守導演的日本科幻暨浪漫電影',
                        actions=[
                            URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                            MessageAction(label='投票', text='我投夏日大作戰一票')
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/2/2d/Castle_of_Cagliostro_poster.png',
                        title='魯邦三世 卡里奧斯特羅城',
                        text='宮崎駿導演的日本動畫動作冒險喜劇電影',
                        actions=[
                            URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96_%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E4%B9%8B%E5%9F%8E'),
                            MessageAction(label='投票', text='我投魯邦三世一票')
                        ]
                    )
                ]
            )
            reply = TemplateMessage(
                alt_text='這是輪播視窗',
                template=carousel_template
            )

        else:
            reply = TextMessage(text='Please type "confirm" or "carousel"!')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

if __name__ == "__main__":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(5000).public_url
    print("👉 請把這個網址填到 LINE Developer Webhook URL：")
    print(f"{public_url}/callback")
    app.run(host="0.0.0.0", port=5000)


👉 請把這個網址填到 LINE Developer Webhook URL：
https://6b3f-34-67-252-214.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:25:26] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:25:35] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:25:38] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:25:54] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:26:03] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:26:09] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 18:26:13] "POST /callback HTTP/1.1" 200 -
